# Let’s embrace WebAssembly!

<img src='images/wasm.png' width=100 align=right>
<img src='images/python.png' width=100 align=right>

EuroPython 2018 - Edinburgh

Almar Klein


<img src='images/pano_ed.jpg' width=100% >


# What is WebAssembly?

<img src='images/wasm.png' align=right width=300>



WebAssembly == WASM

# WASM is an OPEN standard ...

<br>
<div style='display:flex; width:100%; justify-content:space-around; align-items:center;'>
<img src='images/group.png' style='height:200px; width:200px;'>
<img src='images/rocket.png' style='height:100px; width:100px;'>
<img src='images/file.png' style='height:100px; width:100px;'>
<img src='images/check-square.png' style='height:100px; width:100px;'>
</div>
<br>

Collaborative effort by Mozilla, Google, Apple, Microsoft ...


# ... for executable code

<br>
<div style='display:flex; width:100%'>
<img src='images/group.png' style='height:100px; width:100px;'>
<img src='images/rocket.png' style='height:200px; width:200px;'>
<img src='images/file.png' style='height:100px; width:100px;'>
<img src='images/check-square.png' style='height:100px; width:100px;'>
</div>
<br>

It's fast!

<img src='images/assembly1.png' width=600>

<img src='images/assembly2.png' width=600>


# It has a compact binary format

<br>
<div style='display:flex; width:100%'>
<img src='images/group.png' style='height:100px; width:100px;'>
<img src='images/rocket.png' style='height:100px; width:100px;'>
<img src='images/file.png' style='height:200px; width:200px;'>
<img src='images/check-square.png' style='height:100px; width:100px;'>
</div>
<br>

#### And a human readable counterpart:

```wasm
(module
    (type $print (func (param i32))
    (func $main
        (i32.const 42)
        (call $print)
    )
    (start $main)
)
```


# It's safe

<br>
<div style='display:flex; width:100%'>
<img src='images/group.png' style='height:100px; width:100px;'>
<img src='images/rocket.png' style='height:100px; width:100px;'>
<img src='images/file.png' style='height:100px; width:100px;'>
<img src='images/check-square.png' style='height:200px; width:200px;'>
</div>
<br>

Because browsers.

# WebAssembly is coming and it's awesome!

<br>
<div style='display:flex; width:100%'>
<img src='images/group.png' style='height:50px; width:50px;'>
<img src='images/rocket.png' style='height:50px; width:50px;'>
<img src='images/file.png' style='height:50px; width:50px;'>
<img src='images/check-square.png' style='height:50px; width:50px;'>
</div>
<br><br>

<center>
<img src='images/so-much-awesome-meme.jpg' width=460px>
</center>


# WebAssembly adoption

# Lua community

Let's write web apps in Lua !!

<center><img src='images/cat_scream.jpg' width=500></center>

# Rust community

Let's use Rust for everything !!

<center><img src='images/cat_happy.jpg' width=400></center>



# C++ community

We can now  write web apps in C++ ...


<center><img src='images/cat_evil.jpg' width=500px></center>




# JavaScript community


Will this end our suffering?

<center><img src='images/cat_surprised.jpg' width=500></center>

Will this end our monopoly?

# Python community

... what is this WebAssembly thing?

<center><img src='images/cat_bored.jpg' width=500px></center>



# WASM may not be obvious for Python

... Because Python is an *interpreted* language

# Three use-cases how we can embrace WASM

In [ ]:
from ppci import wasm

# Use case 1: Compile a Python interpreter



<center><img src='images/py_to_browser.png' width=800px></center>

# Examples


* Pyodide: compiles CPython + numpy/pandas/matplotlib, to run in the browser
* PyPyJS
* RustPython: Python interpreter written in Rust



Note: Python code is still run in a VM!

# Use case 2: Compile a subset of Python to WASM

<center><img src='images/pysnippet_to_wasm.png' width=800px></center>

In [ ]:
@wasm.wasmify
def find_prime(nth):
    n = 0
    i = -1       
    while n < nth:
        i = i + 1        
        if i <= 1:
            continue  # nope
        elif i == 2:
            n = n + 1
        else:
            gotit = 1
            for j in range(2,  i//2+1):
                if i % j == 0:
                    gotit = 0
                    break
            if gotit == 1:
                n = n + 1    
    return i

In [ ]:
%time find_prime(1000)

# Run in JS

In [ ]:
from ppci.lang.python import python_to_wasm

def main():
    print(find_prime(1000))

m = python_to_wasm(main, find_prime)

In [ ]:
wasm.run_wasm_in_notebook(m)

wasm.run_wasm_in_node(m) and wasm.run_wasm_in_notebook(m)

# Compile a subset of Python to WASM

* Write code to run on the web
* Write code to run fast
* Binaries are cross-platform!

Note: 

* The python-to-wasm compiler is just a POC!
* Assumes a (reliable) wasm-to-native compiler

# Use case 3: Python as a platform to bind and run WASM modules
    
... and allow that code to call into Python functions

<center><img src='images/py_as_platform.png' width=700px></center>

# Rocket game

<center>
    <!-- <a href='https://thread-safe.nl/rocket/' target='new'> -->
    <a href='rocket.html' target='new'>
    <img src='images/github_rocket_wasm.png' width=900>
    </a>
</center>

# Single binary WASM file (58 KB)

<center>    
    <img src='images/github_rocket_wasm_html.png' width=600>
</center>



In [ ]:
from ppci import wasm

m = wasm.Module(open(r'wasm/rocket.wasm', 'rb'))
m

In [ ]:
m.show_interface()

<center>
<img src='images/github_rocket_wasm_js.png' width=1000>
</center>

<center>
<img src='images/rocket_in_js.png' width=800>
</center>

<center>
<img src='images/rocket_in_py.png' width=800>
</center>

In [ ]:
class PythonRocketGame:
    
    # ...
    
    def wasm_sin(self, a:float) -> float:
        return math.sin(a)
    
    def wasm_cos(self, a:float) -> float:
        return math.cos(a)
    
    def wasm_Math_atan(self, a:float) -> float:
        return math.atan(a)
    
    def wasm_clear_screen(self) -> None:
        # ...
    
    def wasm_draw_bullet(self, x:float, y:float) -> None:
        # ...
    
    def wasm_draw_enemy(self, x:float, y:float) -> None:
        # ...
    
    def wasm_draw_particle(self, x:float, y:float, a:float) -> None:
        # ...
    
    def wasm_draw_player(self, x:float, y:float, a:float) -> None:
       # ...
    
    def wasm_draw_score(self, score:float) -> None:
        # ...

# Run Rocket in Python with Qt

In [ ]:
from rocket_qt import QtRocketGame
game = QtRocketGame()

In [ ]:
game.run()

# Run Rocket in Python with prompt_toolkit

Over SSH :)

# This game is not that hard to play ...

## Let's make an AI!

In [ ]:
#print(open('wasm/ai2.c', 'rt').read())

In [ ]:
from ppci import wasm
ai2 = wasm.Module(open('wasm/ai2.wasm', 'rb'))

In [ ]:
ai2.show_interface()

In [ ]:
from rocket_ai import AiRocketGame
game = AiRocketGame(ai2)
game.run()


# Wrapping up ...

## WASM is coming, and its awesome!

<img src='images/wasm.png' width=150 align=right>

* Open, low-level, fast, compact and safe
* Already works in most browsers
* Not limited to the web



<img src='images/python.png' width=150 align=right>

## We Pythonista's should embrace it!

* E.g. run a Python VM in the browser
* E.g. compile subset of Python to fast, crossplatform code
* E.g. use Python as a platform to bind and execute WASM modules

## Thanks!